In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pymssql

In [2]:
positions = pd.read_csv("files/OpenPosition_3_19_2022.csv")
positions = positions[["Symbol", "LastPrice"]]
positions.columns = ["ticker", "price"]
positions = positions.set_index("ticker")
positions = positions.squeeze()

In [3]:
server = 'fs.rice.edu'
database = 'stocks'
username = 'stocks'
password = '6LAZH1'

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database 
conn = create_engine(string).connect()

In [6]:
x = pd.read_sql("select max(date) from sep", conn)
maxdate= str(x.iloc[0, 0])
string = "select ticker, close_ from sep where date=" + "'" + maxdate + "'"
prices = pd.read_sql(string, conn)
prices.columns = ["ticker", "price"]
prices = prices.set_index("ticker")
prices = prices.squeeze()

In [7]:
prices = pd.concat((positions, prices))
duplicates = prices.index.duplicated()
prices = prices[~duplicates]
prices = prices.sort_index()
prices.to_excel("files/prices.xlsx")